In [ ]:
import panel as pn
import altair as alt
from altair import datum
import pandas as pd
from vega_datasets import data
import datetime as dt
import yahoo_fin
from yahoo_fin.stock_info import *

def get_data_for_ticker(ticker_name):
    from2017 = get_data(ticker_name, start_date = '01/01/2017')
    from2017['symbol'] = from2017['ticker']
    from2017['price'] = from2017['close']
    return from2017

In [ ]:
#link gsheet to jupyter notebook

import gspread
from oauth2client.service_account import ServiceAccountCredentials

def get_transaction_data_SGD():
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('employees_secret.json', scope)
    client = gspread.authorize(creds)
    sheet = client.open("Mew's Stock Portfolio Tracker").worksheet("Transactions SGD")
    tracker = sheet.get_all_records()
    sheet_data = pd.DataFrame(tracker)
    transaction_data_SGD = sheet_data[sheet_data['Stock'] != '']
    return transaction_data_SGD

def get_transaction_data_USD():
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('employees_secret.json', scope)
    client = gspread.authorize(creds)
    sheet = client.open("Mew's Stock Portfolio Tracker").worksheet("Transactions USD")
    tracker = sheet.get_all_records()
    sheet_data = pd.DataFrame(tracker)
    transaction_data_USD = sheet_data[sheet_data['Stock'] != '']
    return transaction_data_USD

transaction_data_SGD = get_transaction_data_SGD()
transaction_data_USD = get_transaction_data_USD()

In [ ]:
def get_cumulative_value(transaction_data, ticker_name, stock_name):
    buy_sell_data = transaction_data[transaction_data['Type']!='Div']
    buy_sell_data = buy_sell_data[buy_sell_data['Stock'] == stock_name]
    buy_sell_data = buy_sell_data[['Date', 'Cumulative Units']]
    buy_sell_data['Date'] = pd.to_datetime(buy_sell_data['Date'])
    buy_sell_data.set_index(keys='Date', inplace=True)
    date_range = pd.date_range(start='6/28/2017', end=dt.datetime.today())
    buy_sell_data_new = buy_sell_data.reindex(index=date_range, labels='Date', method='ffill')
    buy_sell_data_new['Price'] = get_data_for_ticker(ticker_name)['price']
    buy_sell_data_new.fillna(inplace=True, method='ffill')
    buy_sell_data_new['Value'] = buy_sell_data_new['Cumulative Units'] * buy_sell_data_new['Price']
    return buy_sell_data_new.fillna(0)

get_cumulative_value(transaction_data_USD, 'iwda.l', 'iShares Core MSCI World UCITS ETF')

In [ ]:
import forex_python
from forex_python.converter import CurrencyRates
c = CurrencyRates()
c.get_rate('USD', 'SGD')

In [ ]:
ticker_to_name_mapping_SGD = {
    'jyeu.si': 'Lendlease REIT',
    'g3b.si': 'Nikko AM STI ETF',
    'a35.si': 'ABF Singapore Bond Index Fund',
    'c52.si': 'ComfortDelGro',
    'a17u.si': 'Ascendas REIT',
}
ticker_to_name_mapping_USD = {
    'googl': 'Alphabet Inc. Class A',
    'gld': 'SPRD Gold Trust',
}

sgd, usd = pd.Series([]), pd.Series([])
for ticker, name in ticker_to_name_mapping_SGD.items():
    sgd = sgd.add(get_cumulative_value(transaction_data_SGD, ticker, name)['Value'], fill_value=0)
for ticker, name in ticker_to_name_mapping_USD.items():
    usd = usd.add(get_cumulative_value(transaction_data_USD, ticker, name)['Value'], fill_value=0)
port_value = pd.DataFrame({'Portfolio Value (SGD)': sgd, 'Portfolio Value (USD)': usd})
port_value['Rate'] = c.get_rate('USD', 'SGD')
port_value['Portfolio Value (Total SGD)'] = port_value['Portfolio Value (SGD)'] + port_value['Portfolio Value (USD)'] * port_value ['Rate']
port_value

In [ ]:
alt.renderers.enable('default')
pn.extension('vega')

# create list of options
tickers = ['SGD', 'USD', 'Total SGD']
# this creates the dropdown widget
ticker_widget = pn.widgets.Select(name='Portfolio', options=tickers)
# this creates the date range slider
date_range_slider = pn.widgets.DateRangeSlider(
    name='Date Range',
    start=dt.datetime(2017, 1, 1), end=dt.datetime.today(),
    value=(dt.datetime(2017, 1, 1), dt.datetime.today())
)
title = '### Portfolio Value Dashboard'
subtitle = 'This dashboard allows you to select a date range to see portfolio value.'

In [ ]:
@pn.depends(ticker_widget.param.value, date_range_slider.param.value)
def get_plot_port_value(ticker, date_range):
    df = port_value
    df['Date'] = df.index
    start_date = date_range_slider.value[0]
    end_date = date_range_slider.value[1]
    mask = (df['Date'] > start_date) & (df['Date'] <= end_date)
    df = df.loc[mask]
    y_label = 'Portfolio Value ({})'.format(ticker)
    chart = alt.Chart(df).mark_line(color="#0c1944").encode(
        x='Date', y=y_label, tooltip=alt.Tooltip(['Date', y_label]))
    return chart

port_value_dashboard = pn.Row(pn.Column(title, subtitle, ticker_widget, date_range_slider), get_plot_port_value)
port_value_dashboard.servable()